In [1]:
import pandas as pd
import random
import numpy as np
from ast import literal_eval

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

In [2]:
df = pd.read_table('data_with_matches.csv', sep=',', converters={"languages": literal_eval, "matches": literal_eval})
mlb = MultiLabelBinarizer()

# Combine languages and help_type
df['features'] = [a+[b] for a,b in zip(df['languages'],df['help_type'])]

# Spread features into columns with binary values
df_feature_separate = pd.DataFrame(mlb.fit_transform(df['features']),columns=mlb.classes_, index=df.index)
for feature_col in df_feature_separate:
    df[feature_col] = df_feature_separate[feature_col]
# save it to pass right columns to split
feature_classes = mlb.classes_

X = df[['intent', *feature_classes]]
y = pd.DataFrame(MultiLabelBinarizer().fit_transform(df['matches']))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [3]:
model = LinearRegression()
model.fit(X_train, y_train)
# model.score(X_test, y_test)

def predict_matches(user_features):
    prediction = model.predict([user_features])
    scores = list(enumerate(prediction[0]))
    sorted_scores = sorted(scores, key = lambda x:x[1], reverse = True)

    sorted_scores[0:100]
    return [x[0] for x in sorted_scores[0:100]]

# Example of how a user would get predicted matches
matches = predict_matches([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0])
matches

[162,
 227,
 250,
 307,
 322,
 405,
 438,
 511,
 625,
 796,
 806,
 827,
 843,
 866,
 1067,
 1430,
 1461,
 1472,
 1572,
 1578,
 1620,
 1653,
 1750,
 1958,
 2022,
 2032,
 2096,
 2190,
 2221,
 2389,
 2478,
 2661,
 2857,
 2903,
 2936,
 2986,
 127,
 128,
 267,
 375,
 765,
 834,
 882,
 891,
 930,
 1001,
 1032,
 1115,
 1203,
 1256,
 1298,
 1398,
 1542,
 1592,
 1642,
 1703,
 1722,
 1762,
 1822,
 1893,
 1996,
 2216,
 2290,
 2339,
 2341,
 2432,
 2561,
 2787,
 2847,
 2883,
 130,
 182,
 240,
 358,
 382,
 492,
 504,
 568,
 672,
 703,
 764,
 791,
 926,
 1173,
 1208,
 1242,
 1341,
 1355,
 1426,
 1488,
 1524,
 1605,
 1680,
 1695,
 1782,
 1816,
 2006,
 2009,
 2494,
 2915]

In [4]:
df.loc[[162,227]]

,id,age,interests,languages,intent,help_type,matches,features,advice,english,forum,german,hangouts,market place,meet people,russian,things to do,ukrainian
162,162,69,community,[english],0,advice,"[10, 42, 81, 88, 91, 104, 106, 121, 122, 143, ...","[english, advice]",1,1,0,0,0,0,0,0,0,0
227,227,19,literature,[english],0,advice,"[10, 42, 81, 88, 91, 104, 106, 121, 122, 143, ...","[english, advice]",1,1,0,0,0,0,0,0,0,0
